In [4]:
import pandas as pd
from sqlalchemy import create_engine 
import pymysql.cursors
import os
import getpass

In [5]:
pw = os.getenv('mysql_mo')

In [6]:
pw = getpass.getpass()

In [7]:
#engine = sqlalchemy.create_engine(f"mysql://{username}:{password}@{host}:{port}")

connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [8]:
with engine.connect() as conn:
    conn.execute(f"CREATE DATABASE IF NOT EXISTS lab_test")

In [9]:
os.getcwd()

'c:\\Users\\marci\\dev\\DAFT_0116_OTALORAMarcia\\Module_1\\Project_Week_3_Data-Preparation\\scripts'

In [10]:
df = pd.read_csv('../output/clean_db.csv')

In [11]:
df

,provincia,comunidad_autonoma,titulo,precio,habitaciones,metros,total_inmuebles_p_comunidad,ascensor,additional_info,extract_address,type_property_main,type_property_sec,precio_sqm,outliers_IQR,outliers_sigma
0,Almeria,andalucia,Casa o chalet independiente en Ciudad Quesada,900.0,2.0,140.0,1345,-1,-1,Ciudad Quesada,casa,indep,6.428571,0,0
1,Almeria,andalucia,Casa o chalet independiente en Playa de Ponien...,2200.0,3.0,278.0,1345,-1,-1,"Playa de Poniente, Benidorm",casa,indep,7.913669,0,0
2,Almeria,andalucia,Casa o chalet independiente en Urbanització Co...,799.0,3.0,75.0,1345,-1,-1,"Urbanització Copet, Coloma, La Nucia",casa,indep,10.653333,0,0
3,Almeria,andalucia,Casa o chalet independiente en avenida las Gol...,800.0,3.0,190.0,1345,-1,-1,"avenida las Golondrinas, Arboleas",casa,indep,4.210526,0,0
4,Almeria,andalucia,"Casa o chalet independiente en Atarazanas, 48,...",550.0,2.0,70.0,1345,-1,-1,"Atarazanas, 48, Cabo de Gata",casa,indep,7.857143,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46810,Vizcaya,pais vasco,"Ático en Casco Viejo, Bilbao",830.0,2.0,50.0,1465,-1,-1,"Casco Viejo, Bilbao",ático,NaN,16.600000,0,0
46811,Vizcaya,pais vasco,"Ático en calle Pérez Galdós, 20, Sabino Arana-...",1400.0,2.0,90.0,1465,-1,-1,"calle Pérez Galdós, 20, Sabino Arana-Jesuitas,...",ático,NaN,15.555556,0,0
46812,Vizcaya,pais vasco,"Ático en cuesta del Hospital Bajo Kalea, 6, La...",1000.0,3.0,115.0,1465,-1,-1,"cuesta del Hospital Bajo Kalea, 6, Las Viñas, ...",ático,NaN,8.695652,0,0
46813,Vizcaya,pais vasco,"Ático en Zabala, Bilbao",980.0,2.0,70.0,1465,-1,-1,"Zabala, Bilbao",ático,NaN,14.000000,0,0


In [12]:
df.columns

Index(['provincia', 'comunidad_autonoma', 'titulo', 'precio', 'habitaciones',
       'metros', 'total_inmuebles_p_comunidad', 'ascensor', 'additional_info',
       'extract_address', 'type_property_main', 'type_property_sec',
       'precio_sqm', 'outliers_IQR', 'outliers_sigma'],
      dtype='object')

In [13]:
df.to_sql('spain_rent',engine, 'lab_test', if_exists='replace', index=False)

46815

In [40]:
sql = """
select comunidad_autonoma, provincia, AVG(precio_sqm) as price_sqm_mean
from lab_test.spain_rent
group by comunidad_autonoma,provincia
order by price_sqm_mean DESC LIMIT 10
"""

In [41]:
new_df = pd.read_sql_query(sql, engine)

In [42]:
new_df

,comunidad_autonoma,provincia,price_sqm_mean
0,cataluña,Barcelona,20.158333
1,baleares,Islas Baleares,18.961605
2,pais vasco,Guipuzcoa,18.434131
3,madrid,Madrid,16.667752
4,cataluña,Gerona,15.776677
5,andalucia,Malaga,15.361786
6,canarias,Las Palmas de Gran Canaria,13.667355
7,pais vasco,Vizcaya,13.063586
8,cantabria,Cantabria,12.814137
9,comunidad valenciana,Valencia,12.457392


In [28]:
sql2 = """
select type_property_main as property_type, AVG(precio_sqm) as price_sqm_mean
from lab_test.spain_rent
group by property_type
order by price_sqm_mean DESC
"""

In [29]:
df_meanprice_type = pd.read_sql_query(sql2, engine)

In [30]:
df_meanprice_type

,property_type,price_sqm_mean
0,ático,12.616615
1,finca,12.521392
2,cortijo,12.322562
3,casa,12.128289
4,chalet,11.436644
5,dúplex,10.934916
6,piso,10.835952
7,torre,10.051387
8,masía,9.861096
9,palacio,3.348999


In [37]:
sql3 = """
select comunidad_autonoma, provincia, type_property_main as property_type, AVG(precio_sqm) as price_sqm_mean
from lab_test.spain_rent
group by comunidad_autonoma, provincia, property_type
order by price_sqm_mean DESC LIMIT 10
"""

In [38]:
df_zone_meanprice = pd.read_sql_query(sql3, engine)

In [39]:
df_zone_meanprice

,comunidad_autonoma,provincia,property_type,price_sqm_mean
0,andalucia,Cadiz,cortijo,27.838095
1,cataluña,Barcelona,ático,25.210009
2,cantabria,Cantabria,masía,23.125000
3,baleares,Islas Baleares,casa,21.759202
4,baleares,Islas Baleares,dúplex,20.598046
5,cataluña,Barcelona,piso,20.079035
6,andalucia,Malaga,casa,19.752266
7,baleares,Islas Baleares,chalet,19.646635
8,baleares,Islas Baleares,ático,19.364664
9,cataluña,Barcelona,dúplex,19.326861


In [51]:
sql4 = """
select comunidad_autonoma, provincia, type_property_main as property_type, AVG(precio_sqm) as price_sqm_mean
from lab_test.spain_rent
where precio_sqm is not null
group by comunidad_autonoma, provincia, property_type
order by price_sqm_mean ASC LIMIT 10
"""

In [52]:
df_zone_meanprice_cheap = pd.read_sql_query(sql4, engine)

In [53]:
df_zone_meanprice_cheap

,comunidad_autonoma,provincia,property_type,price_sqm_mean
0,aragon,Teruel,caserón,0.466667
1,castilla y leon,Valladolid,castillo,0.500000
2,extremadura,Caceres,palacio,1.130000
3,castilla la mancha,Albacete,finca,1.250000
4,cataluña,Gerona,palacio,1.463415
5,castilla y leon,Leon,finca,1.601351
6,castilla la mancha,Ciudad Real,caserón,1.714286
7,castilla y leon,Leon,masía,1.750000
8,aragon,Zaragoza,finca,1.944444
9,andalucia,Jaen,caserón,2.641509
